# Setup

In [104]:
# Load packages 
import re
import ast
import time
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt

In [105]:
%reload_ext autoreload
%autoreload 2

In [106]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [107]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "project"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import col
sqlContext = SQLContext(sc)

# Read in data 

In [108]:
df = sqlContext.read.format("csv") \
               .option("inferSchema", "true") \
               .option("header", "false") \
               .option("delimiter", "\t")\
               .load("data/sample_training.txt") #Adjust path as needed

df = df.withColumnRenamed("_c0", "CTR") \
       .withColumnRenamed("_c1", "NumVar1") \
       .withColumnRenamed("_c2", "NumVar2") \
       .withColumnRenamed("_c3", "NumVar3") \
       .withColumnRenamed("_c4", "NumVar4") \
       .withColumnRenamed("_c5", "NumVar5") \
       .withColumnRenamed("_c6", "NumVar6") \
       .withColumnRenamed("_c7", "NumVar7") \
       .withColumnRenamed("_c8", "NumVar8") \
       .withColumnRenamed("_c9", "NumVar9") \
       .withColumnRenamed("_c10", "NumVar10") \
       .withColumnRenamed("_c11", "NumVar11") \
       .withColumnRenamed("_c12", "NumVar12") \
       .withColumnRenamed("_c13", "NumVar13") \
        .withColumnRenamed("_c14", "NumVar14") \
        .withColumnRenamed("_c15", "NumVar15") \
        .withColumnRenamed("_c16", "NumVar16") \
        .withColumnRenamed("_c17", "NumVar17") \
        .withColumnRenamed("_c18", "NumVar18") \
        .withColumnRenamed("_c19", "NumVar19") \
        .withColumnRenamed("_c20", "NumVar20") \
        .withColumnRenamed("_c21", "NumVar21") \
        .withColumnRenamed("_c22", "NumVar22") \
        .withColumnRenamed("_c23", "NumVar23") \
        .withColumnRenamed("_c24", "NumVar24") \
        .withColumnRenamed("_c25", "NumVar25") \
        .withColumnRenamed("_c26", "NumVar26") \
        .withColumnRenamed("_c27", "NumVar27") \
        .withColumnRenamed("_c28", "NumVar28") \
        .withColumnRenamed("_c29", "NumVar29") \
        .withColumnRenamed("_c30", "NumVar30") \
        .withColumnRenamed("_c31", "NumVar31") \
        .withColumnRenamed("_c32", "NumVar32") \
        .withColumnRenamed("_c33", "NumVar33") \
        .withColumnRenamed("_c34", "NumVar34") \
        .withColumnRenamed("_c35", "NumVar35") \
        .withColumnRenamed("_c36", "NumVar36") \
        .withColumnRenamed("_c37", "NumVar37") \
        .withColumnRenamed("_c38", "NumVar38") \
        .withColumnRenamed("_c39", "NumVar39")


#df.select("*").show(5)
#df.select("CTR", "NumVar1", "NumVar2", "NumVar3", "NumVar4", "NumVar5", "NumVar6", "NumVar7", "NumVar8", "NumVar9", "NumVar10", "NumVar11", "NumVar12", "NumVar13").show()




In [109]:
subset_ctr1 = df.filter(col("CTR") == 1) #get all of the CTR == 1 

### A few (hopefully useful :-) ) pointers

In [111]:
#Compute total count of 1 in target column
#in the line below one could simply do .count(), since you've filtered previously. However,
#I feel the alternative below is safer, if somebody changes the code later on and removes 
#the previous filtering...
count_1 = subset_ctr1.groupBy().sum('CTR').collect()[0][0]
count_1

587833

In [112]:
#Select columns for computation
compute_cols = ['NumVar20','NumVar28']

#The one below counts the number of unique elements in the columns
from pyspark.sql.functions import col
for col in subset_ctr1[compute_cols]:
    print('This column has: ' + str(subset_ctr1.select(col).distinct().count()) + ' unique elements.')

This column has: 10182 unique elements.
This column has: 8043 unique elements.


In [113]:
#Select columns for computation
compute_cols = ['NumVar20','NumVar28']

#The one below partitions the column by unique elements and counts the number of rows
#for each unique element
from pyspark.sql.functions import col, desc
for col in subset_ctr1[compute_cols]:
    df = subset_ctr1.groupBy([col]).count().sort(desc('count')).cache()
    df.show()

+--------+-----+
|NumVar20|count|
+--------+-----+
|1c86e0eb|18899|
|468a0854|11136|
|7195046d| 8185|
|407438c8| 6152|
|dc7659bd| 6119|
|8379baa1| 5037|
|5e64ce5f| 4937|
|fe4dce68| 4900|
|81bb0302| 4608|
|5d87968e| 4055|
|ad3508b1| 3939|
|d2dbdfe6| 3847|
|90a2c015| 3372|
|3f4ec687| 3286|
|38eb9cf4| 3055|
|45e063a0| 2874|
|4aa938fc| 2873|
|ce4f7f55| 2828|
|8f572b5e| 2617|
|7227c706| 2587|
+--------+-----+
only showing top 20 rows

+--------+-----+
|NumVar28|count|
+--------+-----+
|2d0bb053| 7751|
|d345b1a0| 5681|
|46ed0b3c| 5342|
|f6b23a53| 5336|
|10935a85| 5234|
|10040656| 5121|
|36721ddc| 4889|
|f3002fbd| 4742|
|e1ac77f7| 4521|
|cae64906| 4517|
|310d155b| 4333|
|34cce7d2| 4091|
|3628a186| 4078|
|9efd8b77| 3690|
|a9d1ba1a| 3630|
|25753fb1| 3596|
|0af7c64c| 3539|
|af56328b| 3360|
|14108df6| 3246|
|e8d4033b| 3224|
+--------+-----+
only showing top 20 rows



In [114]:
#Select columns for computation
compute_cols = ['NumVar20','NumVar28']

#The one below partitions the column by unique elements and counts the number of rows
#for each unique element, then divides the number of rows for each unique element
#by the total number of 1s (again, this should be the frequency you are referring to?)
from pyspark.sql.functions import col, desc
for col in subset_ctr1[compute_cols]:
    df = subset_ctr1.groupBy([col]).count().sort(desc('count')).cache()
    df = df.withColumn('count_frequ', df['count']/count_1)
    df.show()

+--------+-----+--------------------+
|NumVar20|count|         count_frequ|
+--------+-----+--------------------+
|1c86e0eb|18899| 0.03215028758167711|
|468a0854|11136|0.018944155908225634|
|7195046d| 8185| 0.01392402263908287|
|407438c8| 6152|0.010465557394702237|
|dc7659bd| 6119|0.010409419001655233|
|8379baa1| 5037|0.008568760175083739|
|5e64ce5f| 4937|0.008398643832517058|
|fe4dce68| 4900|0.008335700785767387|
|81bb0302| 4608|0.007838961065472676|
|5d87968e| 4055|0.006898217691078929|
|ad3508b1| 3939|0.006700882733701579|
|d2dbdfe6| 3847|0.006544375698540232|
|90a2c015| 3372|0.005736323071348...|
|3f4ec687| 3286| 0.00559002301674115|
|38eb9cf4| 3055|0.005197054265412...|
|45e063a0| 2874|0.004889143685366422|
|4aa938fc| 2873|0.004887442521940755|
|ce4f7f55| 2828|0.004810890167785749|
|8f572b5e| 2617|0.004451944684970051|
|7227c706| 2587|0.004400909782200047|
+--------+-----+--------------------+
only showing top 20 rows

+--------+-----+--------------------+
|NumVar28|count|        

In [115]:
#Select columns for computation
compute_cols = ['NumVar20','NumVar28']

#The one below partitions the column by unique elements and counts the number of rows
#for each unique element, then divides the number of rows for each unique element
#by the total number of 1s (again, this should be the frequency you are referring to?)
from pyspark.sql.functions import col, desc
from pyspark.sql import window
from pyspark.sql import functions as F

win_spec = (window.Window
                  .partitionBy()
                  .rowsBetween(window.Window.unboundedPreceding, 0))

#adapted from here: https://stackoverflow.com/questions/34726268/how-to-calculate-cumulative-sum-using-sqlcontext
for col in subset_ctr1[compute_cols]:
    df = subset_ctr1.groupBy([col]).count().sort(desc('count')).cache()
    df = df.withColumn('count_frequ', df['count']/count_1).cache()
    df = df.withColumn('cumsum',F.sum(df.count_frequ).over(win_spec))
    df.show()

+--------+-----+--------------------+-------------------+
|NumVar20|count|         count_frequ|             cumsum|
+--------+-----+--------------------+-------------------+
|1c86e0eb|18899| 0.03215028758167711|0.03215028758167711|
|468a0854|11136|0.018944155908225634|0.05109444348990275|
|7195046d| 8185| 0.01392402263908287|0.06501846612898561|
|407438c8| 6152|0.010465557394702237|0.07548402352368785|
|dc7659bd| 6119|0.010409419001655233|0.08589344252534309|
|8379baa1| 5037|0.008568760175083739|0.09446220270042682|
|5e64ce5f| 4937|0.008398643832517058|0.10286084653294388|
|fe4dce68| 4900|0.008335700785767387|0.11119654731871126|
|81bb0302| 4608|0.007838961065472676|0.11903550838418393|
|5d87968e| 4055|0.006898217691078929|0.12593372607526288|
|ad3508b1| 3939|0.006700882733701579|0.13263460880896447|
|d2dbdfe6| 3847|0.006544375698540232| 0.1391789845075047|
|90a2c015| 3372|0.005736323071348...| 0.1449153075788532|
|3f4ec687| 3286| 0.00559002301674115|0.15050533059559434|
|38eb9cf4| 305

In [119]:
from pyspark.sql.functions import col
df.filter(col('cumsum')<0.75).count()

572